<a href="https://colab.research.google.com/github/MVanDenburg92/RGEE_Shiny/blob/main/RGEE_Colab_Scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Pheology RGEE Analysis Google Collab Demo**
<a href="https://colab.research.google.com/github/r-spatial/rgee/blob/examples/rgee_colab.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
<br>
OBS: Consider that **you can not** run **`googledrive`** and **`googleCloudStorageR`** in a non-interactive session. The following functions will not work.
- ee_drive_to_local()
- ee_as_raster(via = "drive")
- ee_as_stars(via = "drive")
- ee_as_sf(via = "drive")
- rgee::ee_gcs_to_local()
- ee_as_raster(via = "gcs")
- ee_as_stars(via = "gcs")
- ee_as_sf(via = "gcs")
- sf_as_ee(via = "gcs_to_asset")
- gcs_to_ee_image
- raster_as_ee
- local_to_gcs- stars_as_ee

In [ ]:
#@title **Install rgee dependencies. It takes aprox. 12 min**
system('sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable')
system('sudo apt-get update')
system('sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev')
system('sudo apt-get install libprotobuf-dev protobuf-compiler libv8-dev libjq-dev')
install.packages('sf')
install.packages('mapview')
install.packages('cptcity')
install.packages('geojsonio')
install.packages('qdapRegex')
install.packages('lubridate')
install.packages('doParallel')
install.packages('foreach')
install.packages('parallel')
install.packages('raster')
install.packages('geojsonio')
install.packages('qdapRegex')
install.packages('lubridate')
install.packages('doParallel')
install.packages('gsubfn')
install.packages('foreach')
remotes:::install_github("rstudio/reticulate")


remotes::install_github("r-spatial/rgee")
remotes::install_github("rforge/greenbrown/pkg/greenbrown/", force = TRUE)

In [ ]:
#@title **help functions**
help <- function(func){
  func <- as.character(substitute(func))
  IRdisplay::display(utils::help(func))
}

ee_help <- function(func){  
  func <- as.character(substitute(func))
  tmp_html <- rgee::ee_help(func)
  IRdisplay::display_html(file = tmp_html)
}

In [ ]:
#@title **embed function**
# Created by cromulent
# https://stackoverflow.com/questions/46770320/error-html-widgets-cannot-be-represented-in-plain-text
ee_embed = function(x, height = 450) {    
    tmp = tempfile(fileext = ".html")
    htmlwidgets::saveWidget(x@map, tmp)
    rawHTML = base64enc::dataURI(mime = "text/html;charset=utf-8", file = tmp)
    IRdisplay::display_html(paste("<iframe src=", rawHTML, "width=100% height=", height, "id=","igraph", "scrolling=","no","seamless=","seamless", "frameBorder=","0","></iframe>", sep = "\""))
    unlink(tmp)
}

### **1. Initialize and Authenticate**

In [ ]:
library(rgee)
ee_Initialize(email = 'miles.vandenburg@gmail.com',display = TRUE)

### **2. Hello world!**

In [ ]:
remotes:::install_github("rstudio/reticulate")

In [ ]:
install.packages('gsubfn')

In [51]:
install.packages("shiny")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘fastmap’




In [ ]:
library(sf)
library(sp)
library(tidyverse)
library(cptcity)
library(mapview)
library(greenbrown)

library(raster)
library(qdapRegex)

library(doParallel)  #Foreach Parallel Adaptor 
library(foreach)
library(parallel)
library(gsubfn)
library(rgee)
#library(reticulate)

In [ ]:
 shape = 'Ecoregion_Small.shp'
band_name = 'EVI'
img_collection = "LANDSAT/LT05/C01/T1_32DAY_EVI"
start = "2000-01-01"
end = "2008-01-01"
 poly2 = matrix(c(-71.91549,42.43691, -71.91142 ,42.43696,  -71.91109, 42.43207, -71.91526, 42.43228, -71.91549, 42.43691),ncol = 2, byrow = TRUE)
            
            # poly3 <- st_polygon(list(poly2))
            
            
            poly2 <-  Polygon(poly2)
            
            Ps1 <-  SpatialPolygons(list(Polygons(list(poly2), ID = "a")), proj4string=CRS("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"))
            poly2 <- as(Ps1, "sf")
            
            
            # 
            # 
            # shapefile <- Ps1
            # 
            # eco_mask2 <- st_read(shapefile,quiet = TRUE) %>% st_transform(4326) 
            # 
            # plot(eco_mask2)
            # 
            eco_mask_ee2 <- sf_as_ee(poly2)
            
            
            ##Bring in the imagery from RGEE
            # 
              s4 <- ee$ImageCollection(img_collection)
  # start <- '2000-01-01'
  # end <- '2008-01-01'
  # 
  s4 <-  s4$filterDate(ee$Date(start),ee$Date(end))$filterBounds(eco_mask_ee2)

  nbrImages_s4 = s4$size() %>% ee$Number()
  
  
  #Mapping the clip function over the collection
  s4 = s4$map(function(image){image$clip(eco_mask_ee2)})
  
  # s4info2 <- s4$getInfo()
  
  #Selecting the EVI band which holds the information
  s4 = s4$select(band_name)


  np = reticulate:::import("numpy")      # Import Numpy 

<center>
<img src="https://user-images.githubusercontent.com/16768318/89118214-486d5200-d469-11ea-8e16-085cc1888df6.png" width=40% >
</center>

In [ ]:
  nimages2 <- s4 %>% ee$FeatureCollection$size() %>% ee$Number()
  ic_date2 <- ee_get_date_ic(s4)
  
  
  s4_img_list <- list()
  latlng2 <- list()
  lats2 <- list()
  lngs2 <- list()
  evi_values4 <- list()
  s4_names <- list()
  
  evi_values5 <- list()
  evi_values6 <- list()

In [ ]:
for(i in seq(nimages2$getInfo())){
    # as.numeric(i)
    py_index <- i - 1
    s4_img <- ee$Image(s4 %>% ee$FeatureCollection$toList(1, py_index) %>% ee$List$get(0))
    s4_name <- s4_img %>% ee_get_date_img()
    s4_names[[i]] <- as.character(s4_name$time_start)
    s4_img <- s4_img$select(band_name)$rename(as.character(s4_name$time_start))
    s4_img_list[[i]] <- ee$Image$pixelLonLat()$addBands(s4_img)
    s4_img_list[[i]] <-  s4_img_list[[i]]$reduceRegion(reducer = ee$Reducer$toList(),
                                                       geometry  = eco_mask_ee2,
                                                       maxPixels = 1e6,
                                                       scale = 30,
                                                       bestEffort = TRUE)
  }

In [ ]:
 func_dos <- function(q) {
    lats2 <- np$array((ee$Array(s4_img_list[[q]]$get("latitude")) %>% ee$Array$getInfo()))
    lngs2 <- np$array((ee$Array(s4_img_list[[q]]$get("longitude")) %>% ee$Array$getInfo()))
    
    data.frame(x = lngs2,y = lats2)
    evi_values4 <- list()
    
    for(index in  seq(nimages2$getInfo())) {
      evi_values4[[index]] <-  ee$List(s4_img_list[[index]]$get(s4_names[[index]]))$getInfo()
      
    }
    
    #Convert list elements of evi_values into num py arrays
    evi_values5 <- lapply(evi_values4,function(x){
      np$array((x))
    })
    
    eviss4 <- evi_values5
    names(eviss4) <- ic_date2$id
    evis_df_s4 <- data.frame(x = lngs2,y = lats2,lapply(eviss4, "length<-", max(lengths(eviss4))))
  }
  s <- func_dos(1)

In [ ]:
evis_df_s4 <- s

evis_df_s4

Full Code to Create Dataframe Output 

In [48]:
shape = 'Ecoregion_Small.shp'
band_name = 'EVI'
img_collection = "LANDSAT/LT05/C01/T1_32DAY_EVI"
start = "2000-01-01"
end = "2008-01-01"

gee_extract <- function(shape = 'Ecoregion_Small.shp', band_name = 'EVI', img_collection = "LANDSAT/LT05/C01/T1_32DAY_EVI", start = "2000-01-01", end = "2008-01-01") {
  
poly2 = matrix(c(-71.91549,42.43691, -71.91142 ,42.43696,  -71.91109, 42.43207, -71.91526, 42.43228, -71.91549, 42.43691),ncol = 2, byrow = TRUE)
            
            # poly3 <- st_polygon(list(poly2))
            
            
            poly2 <-  Polygon(poly2)
            
            Ps1 <-  SpatialPolygons(list(Polygons(list(poly2), ID = "a")), proj4string=CRS("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"))
            poly2 <- as(Ps1, "sf")
            
            
            # 
            # 
            # shapefile <- Ps1
            # 
            # eco_mask2 <- st_read(shapefile,quiet = TRUE) %>% st_transform(4326) 
            # 
            # plot(eco_mask2)
            # 
            eco_mask_ee2 <- sf_as_ee(poly2)
            
            
            ##Bring in the imagery from RGEE
            # 
              s4 <- ee$ImageCollection(img_collection)
  # start <- '2000-01-01'
  # end <- '2008-01-01'
  # 
  s4 <-  s4$filterDate(ee$Date(start),ee$Date(end))$filterBounds(eco_mask_ee2)

  nbrImages_s4 = s4$size() %>% ee$Number()
  
  
  #Mapping the clip function over the collection
  s4 = s4$map(function(image){image$clip(eco_mask_ee2)})
  
  # s4info2 <- s4$getInfo()
  
  #Selecting the EVI band which holds the information
  s4 = s4$select(band_name)
  
  #Get information about the collection 
  # s4info3 <- s4$getInfo()
  
  
  
  
  ## SMALLER REGIONS
  
  #Get setup for the loop.  In case the loop crashes or doesn't work, you must rerun the above chunk first before running this chunk again. 
  
  np = import("numpy")      # Import Numpy 
  
  
  # nimages2 <- s4$size()$getInfo()  
  
  nimages2 <- s4 %>% ee$FeatureCollection$size() %>% ee$Number()
  ic_date2 <- ee_get_date_ic(s4)
  
  
  s4_img_list <- list()
  latlng2 <- list()
  lats2 <- list()
  lngs2 <- list()
  evi_values4 <- list()
  s4_names <- list()
  
  evi_values5 <- list()
  evi_values6 <- list()
  
  # nimages2$getInfo()
  
  
  for(i in seq(nimages2$getInfo())){
    # as.numeric(i)
    py_index <- i - 1
    s4_img <- ee$Image(s4 %>% ee$FeatureCollection$toList(1, py_index) %>% ee$List$get(0))
    s4_name <- s4_img %>% ee_get_date_img()
    s4_names[[i]] <- as.character(s4_name$time_start)
    s4_img <- s4_img$select(band_name)$rename(as.character(s4_name$time_start))
    s4_img_list[[i]] <- ee$Image$pixelLonLat()$addBands(s4_img)
    s4_img_list[[i]] <-  s4_img_list[[i]]$reduceRegion(reducer = ee$Reducer$toList(),
                                                       geometry  = eco_mask_ee2,
                                                       maxPixels = 1e6,
                                                       scale = 30,
                                                       bestEffort = TRUE)
  }
  
  func_dos <- function(q) {
    lats2 <- np$array((ee$Array(s4_img_list[[q]]$get("latitude")) %>% ee$Array$getInfo()))
    lngs2 <- np$array((ee$Array(s4_img_list[[q]]$get("longitude")) %>% ee$Array$getInfo()))
    
    data.frame(x = lngs2,y = lats2)
    evi_values4 <- list()
    
    for(index in  seq(nimages2$getInfo())) {
      evi_values4[[index]] <-  ee$List(s4_img_list[[index]]$get(s4_names[[index]]))$getInfo()
      
    }
    
    #Convert list elements of evi_values into num py arrays
    evi_values5 <- lapply(evi_values4,function(x){
      np$array((x))
    })
    
    eviss4 <- evi_values5
    names(eviss4) <- ic_date2$id
    evis_df_s4 <- data.frame(x = lngs2,y = lats2,lapply(eviss4, "length<-", max(lengths(eviss4))))
  }
  s <- func_dos(1)
  
}

In [49]:
evis_df_s4 <- s
evis_df_s4


x,y,LANDSAT.LT05.C01.T1_32DAY_EVI.20000101,LANDSAT.LT05.C01.T1_32DAY_EVI.20000202,LANDSAT.LT05.C01.T1_32DAY_EVI.20000305,LANDSAT.LT05.C01.T1_32DAY_EVI.20000406,LANDSAT.LT05.C01.T1_32DAY_EVI.20000508,LANDSAT.LT05.C01.T1_32DAY_EVI.20000609,LANDSAT.LT05.C01.T1_32DAY_EVI.20000711,LANDSAT.LT05.C01.T1_32DAY_EVI.20000812,⋯,LANDSAT.LT05.C01.T1_32DAY_EVI.20070101,LANDSAT.LT05.C01.T1_32DAY_EVI.20070202,LANDSAT.LT05.C01.T1_32DAY_EVI.20070306,LANDSAT.LT05.C01.T1_32DAY_EVI.20070407,LANDSAT.LT05.C01.T1_32DAY_EVI.20070509,LANDSAT.LT05.C01.T1_32DAY_EVI.20070610,LANDSAT.LT05.C01.T1_32DAY_EVI.20070712,LANDSAT.LT05.C01.T1_32DAY_EVI.20070813,LANDSAT.LT05.C01.T1_32DAY_EVI.20070914,LANDSAT.LT05.C01.T1_32DAY_EVI.20071219
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-71.91503,42.43233,0.07541332,0.09110306,0.0551460,0.1912489,0.5679646,0.7401720,0.5707733,0.6586844,⋯,0.2820837,0.1610811,0.01084119,0.3176929,0.7558748,0.6021477,-0.055817375,0.7307673,0.4313671,NA
-71.91476,42.43233,0.17284705,0.07043627,0.1205535,0.1385721,0.6286677,0.6305169,0.5303880,0.5655420,⋯,0.2502909,0.1799942,0.16080018,0.2494227,0.6528083,0.6175965,0.009251597,0.5938833,0.3782148,NA
-71.91450,42.43233,0.38943305,0.07244610,0.3004876,0.1323357,0.6286677,0.6690960,0.5375242,0.6187152,⋯,0.3788692,0.3300306,0.31412829,0.2956242,0.6616833,0.6843576,0.060556698,0.6095217,0.4099924,NA
-71.91423,42.43233,0.45205441,0.07675098,0.3723714,0.1507066,0.6607197,0.7570703,0.5634595,0.6489832,⋯,0.4493466,0.5059334,0.39539435,0.3558981,0.6350040,0.7066953,0.107249141,0.6177863,0.3831436,NA
-71.91396,42.43233,0.45205441,0.07675098,0.3723714,0.1507066,0.6823782,0.7570703,0.5634595,0.6489832,⋯,0.4493466,0.5059334,0.39539435,0.3558981,0.6350040,0.7066953,0.107249141,0.6177863,0.3831436,NA
-71.91369,42.43233,0.45874489,0.09662261,0.3878071,0.1509755,0.7799154,0.8231526,0.5970171,0.6472558,⋯,0.4006685,0.4194448,0.45125424,0.3433393,0.6770254,0.7670699,0.114858004,0.6448599,0.4050553,NA
-71.91342,42.43233,0.45921946,0.10295086,0.3765071,0.1474364,0.7799154,0.8603828,0.6074284,0.6439896,⋯,0.3880381,0.3489832,0.28760644,0.3076609,0.6771980,0.7334920,-0.249438007,0.5688264,0.4143150,NA
-71.91315,42.43233,0.42175356,0.07753677,0.3341011,0.1605717,0.7565054,0.7045317,0.5784753,0.5533564,⋯,0.2246084,0.2222574,0.17932729,0.2372131,0.5337719,0.5125833,0.014753540,0.4117151,0.3489907,NA
-71.91288,42.43233,0.42175356,0.07753677,0.3341011,0.1605717,0.6828149,0.7045317,0.5784753,0.5533564,⋯,0.2246084,0.2222574,0.17932729,0.2372131,0.5337719,0.5125833,0.014753540,0.4117151,0.3489907,NA


Raster Brick Function and Code

In [ ]:
dataframe_processing <- function(){

pos <- grep(pattern = "NA", evis_df_s4)
while(length(pos)>0){
  for(t in pos){
    if(t < ncol(evis_df_s4)){
      evis_df_s4[t] <- (evis_df_s4[t+1]-evis_df_s4[t-1])/2
    }else
      evis_df_s4[t] <- evis_df_s4[ncol(evis_df_s4)-1]
  }
  pos <- grep(pattern = "NA", x = evis_df_s4)
  
}


coords3 <- evis_df_s4 %>% dplyr::select(x, y) 

evis_df_s4 <- evis_df_s4 %>% select(-c(x, y))

evis_df_s4[evis_df_s4 < 0] = 0

evis_df_s4_2 <- data.frame(coords3,evis_df_s4)

#add reformatted dates

colnames_df <-  names(evis_df_s4_2)

#Extract dates from colummns
dates <- gsubfn:::strapplyc(colnames_df, "[0-9]{8,}", simplify = TRUE)
dates <- lubridate:::ymd(dates[-c(1:2)])

names(evis_df_s4_2)[3:length(evis_df_s4_2)] <- as.character(dates)


evis_df_s4_3 <- evis_df_s4_2 %>%
  mutate_all(as.numeric) 


#Extract years from dates 
y2 <- data.frame(Date=(seq(dates[1], dates[length(dates)], by = 'year')))
y2$Year <-year(y2$Date)
years <- y2$Year

xxx <- length(years)*5


all_pixels_2 <- evis_df_s4_2 %>% dplyr:::select(-c(x, y)) %>% arrange() %>% slice(1:nrow(evis_df_s4_2)) %>% t() 


# #create time series 
EVIseries2 <- ts(all_pixels_2, start=c(years[1], 1), end=c(years[length(years)], 12), frequency=12)


#Create empty lists for lapply
Yt_interpolate_m3 <- list()
Yt_interpolate_m4 <- list()
Phen_2000_2007_3 <- list()


#Apply Tspp # time series pre-processing ---interpolating across whole data set.  Use lapply to retain time series information. 
ww <- ncol(EVIseries2)

# system.time(
# Yt_interpolate_m4 <- lapply(seq(ww), function(x){
#   Yt_interpolate_m3[[x]]  <- TsPP(EVIseries2[,x], tsgf=TSGFspline)
# }))

# user  system elapsed 
# 7.65    0.04    8.01 


model.mse <- function(x){
  Yt_interpolate_m3[[x]]  <- greenbrown::TsPP(EVIseries2[,x], tsgf=TSGFspline)
}




model_phen <- function(x){
  Phen_2000_2007_3[[x]] <- greenbrown:::Phenology(Yt_interpolate_m4[[x]], approach="White")
}

UseCores <- detectCores() - 2


clust <- makeCluster(UseCores)
parallel:::clusterEvalQ(cl = clust, library("greenbrown"))
parallel:::clusterExport(cl = clust, c("EVIseries2","Yt_interpolate_m3"), envir=environment())
Yt_interpolate_m4 <- parLapply(clust, seq(ww), model.mse)

# user  system elapsed 
# 0.03    0.01    5.69 



  # clust <- makeCluster(UseCores)
clusterEvalQ(clust, library("greenbrown"))
clusterExport(clust, c('Phen_2000_2007_3', 'Yt_interpolate_m4'),envir=environment())
Phen_2000_2007_3 <- parLapply(clust, seq(ww), model_phen)

#4 cores
# #   user  system elapsed 
# 0.24    0.07   39.80 

#6 cores

# user  system elapsed 
# 0.22    0.03   31.72 


stopCluster(clust)
getDoParWorkers()
registerDoSEQ()
stopImplicitCluster()



#Create empty list containers for loop
sos2 <- list()
eos2 <- list()
los2 <- list()
pop2 <- list()


for(i in seq(ncol(EVIseries2))){
  sos2[[i]] <-  as.numeric( Phen_2000_2007_3[[i]][["sos"]])
  eos2[[i]] <-  as.numeric( Phen_2000_2007_3[[i]][["eos"]])
  los2[[i]] <-  as.numeric( Phen_2000_2007_3[[i]][["los"]])
  pop2[[i]] <-  as.numeric( Phen_2000_2007_3[[i]][["pop"]])
  
}

plot(Yt_interpolate_m4[[1]])

sos2 <- unlist(sos2)
eos2 <- unlist(eos2)
los2 <- unlist(los2)
pop2 <- unlist(pop2)


coords3 <-evis_df_s4_2 %>% dplyr:::select(x, y) 
coords4 <- coords3 %>% slice(rep(1:n(), each = 8))



DF_phenmet_test2 <- data.frame(coords4,sos2, eos2, los2, pop2) %>% cbind(years)

DF_phenmet_testarranged2 <- DF_phenmet_test2 %>% arrange(years)


DF_phenmet_testarranged_sf2 = st_as_sf(DF_phenmet_testarranged2, coords = c("x", "y"), crs = 4326)


year_chars2 <- as.character(years)
#year_chars[1]

separated_rasters2 <- list()
# x <- length(years)

for(g in seq(years)){
  separated_rasters2[[g]] <- lapply(years[g], function(x){DF_phenmet_testarranged2 %>% filter(years == year_chars2[g])})
}

DF_phenmet_testarranged2 %>% filter(years == year_chars2[1])



# 
# for(h in 3:7){
# names(separated_rasters2[[1]][[1]]) <- paste(names(separated_rasters2[[x]][[1]]), separated_rasters2[[x]][[1]][["years"]][[h]], sep = '_')
# }


rasterList2 <- brick()

rasterList2 <- lapply(1:length(separated_rasters2), function(x){rasterFromXYZ(separated_rasters2[[x]][[1]])})

# h <- 3:7

for(kk in 1:length(separated_rasters2)){
  lapply(3:7, function(x){
     names(separated_rasters2[[kk]][[1]]) <- paste(names(separated_rasters2[[kk]][[1]]), separated_rasters2[[kk]][[1]][["years"]][[x]], sep = '_')})
}
# rasterList2[[1]]@data@names


rasterList3 <- brick()

rasterList3 <- lapply(1:length(separated_rasters2), function(x){rasterList2[[x]]@data@names[x] <- paste(rasterList2[[x]]@data@names[x], year_chars2, sep = '_')
rasterList2})


rasterBrick2 <- brick(rasterList2)

  
}

rasterBrick_out <- dataframe_processing()


# xmlSVG({plot(rasterBrick_out

# plots <- lapply(1:5, function(i){
#   xmlSVG({plot(rnorm(50), main=paste0("Iteration ", i))}, standalone = TRUE)
# })
# 
# plotsAsSVG <- sapply(rasterBrick_out, function(sv){
#   paste0("data:image/svg+xml;utf8,",as.character(sv))
# })
# 
# imgs <- plotsAsSVG
# slickR(imgs)

# plot(rasterBrick_out, facets)

library(ggplot2)
plot(rasterBrick_out, maxnl=32)